# Módulo 4: Desarrollar una solución multi-agente

En este ejercicio, crearás una solución multi-agente para el triaje de tickets de soporte técnico. Un agente coordinador utilizará otros agentes especializados para evaluar la prioridad, asignar el equipo y estimar el esfuerzo.

### Cargar librerías y variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Conectar al Cliente de Agentes (Agent Client)

In [ ]:
print("Connecting to Azure AI Agents Client...")
agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True,
    )
)
print("Agents client connected successfully")

### Crear Agentes Especializados

Primero, creamos los tres agentes especializados: uno para priorizar tickets, otro para asignar equipos y un tercero para estimar el esfuerzo.

In [ ]:
# Priority evaluation agent
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Evaluate the urgency of technical tickets for aerospace operations.
Respond with: Critical, High, Medium, or Low with brief explanation.
"""
print("Creating priority agent...")
priority_agent = agents_client.create_agent(
    model=model_deployment,
    name=priority_agent_name,
    instructions=priority_agent_instructions,
)
print(f"Priority agent created (id: {priority_agent.id})")

# Team assignment agent
team_agent_name = "team_agent"
team_agent_instructions = """
Determine which department should handle each ticket: Engineering, Manufacturing, Quality Control, Supply Chain, or Maintenance.
Base your decision on the technical content. Return department name and brief explanation.
"""
print("Creating team assignment agent...")
team_agent = agents_client.create_agent(
    model=model_deployment,
    name=team_agent_name,
    instructions=team_agent_instructions,
)
print(f"Team assignment agent created (id: {team_agent.id})")

# Effort estimation agent
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estimate required effort: Small, Medium, Large, or Extra Large.
Consider aerospace complexity. Return effort level with technical justification.
"""
print("Creating effort estimation agent...")
effort_agent = agents_client.create_agent(
    model=model_deployment,
    name=effort_agent_name,
    instructions=effort_agent_instructions,
)
print(f"Effort estimation agent created (id: {effort_agent.id})")

### Crear Herramientas de Agentes Conectados (Connected Agent Tools)

Para que el agente coordinador pueda invocar a los agentes especializados, creamos herramientas para cada uno.

In [ ]:
print("Creating connected agent tools...")
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, name=priority_agent_name, description="Evaluate ticket priority"
)
team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, name=team_agent_name, description="Determine which team should handle ticket"
)
effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, name=effort_agent_name, description="Estimate effort required to complete ticket"
)
print("Connected agent tools created")

### Crear el Agente Coordinador de Triaje

Este agente orquestará el trabajo de los agentes especializados.

In [ ]:
triage_agent_name = "triage-agent"
triage_agent_instructions = """
You are the technical triage coordinator. Analyze each ticket and use specialist agents to determine:
1. Priority level
2. Department assignment
3. Effort estimation
Provide comprehensive triage summary.
"""
print("Creating triage coordinator agent...")
triage_agent = agents_client.create_agent(
    model=model_deployment,
    name=triage_agent_name,
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0],
    ],
)
print(f"Triage coordinator agent created (id: {triage_agent.id})")

### Usar los agentes para el triaje de un ticket

Introduce un problema de soporte en el `prompt` y ejecuta la celda para que los agentes lo procesen.

In [ ]:
print("Creating thread...")
thread = agents_client.threads.create()
print(f"Thread created (id: {thread.id})")

# Enter support issue here
prompt = "Out-of-specification dimensional tolerances detected on high-pressure turbine blades. Batch Q1-2024-TB-156 affected."

print(f"Sending prompt: '{prompt}'")
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)
print(f"User message created (thread: {thread.id})")

print("\nProcessing triage... please wait.")
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=triage_agent.id)
print(f"Triage run completed with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    print("Retrieving triage results...")
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.text_messages:
            last_msg = message.text_messages[-1]
            print(f"{message.role}:\n{last_msg.text.value}\n")

### Limpieza

Finalmente, eliminamos todos los agentes creados.

In [ ]:
print("Cleaning up agents...")
print(f"Deleting triage agent (id: {triage_agent.id})...")
agents_client.delete_agent(triage_agent.id)
print("Triage agent deleted")

print(f"Deleting priority agent (id: {priority_agent.id})...")
agents_client.delete_agent(priority_agent.id)
print("Priority agent deleted")

print(f"Deleting team assignment agent (id: {team_agent.id})...")
agents_client.delete_agent(team_agent.id)
print("Team assignment agent deleted")

print(f"Deleting effort estimation agent (id: {effort_agent.id})...")
agents_client.delete_agent(effort_agent.id)
print("Effort estimation agent deleted")

print("All agents cleaned up successfully")